# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "human_enhancers_cohn"
TOKENIZER = "kmer"
KMER = 2


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

human_enhancers_cohn 0 kmer 2


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 18
{'TA': 15, 'TT': 14, 'GC': 13, 'TC': 11, 'AA': 4, 'AG': 9, 'CC': 6, 'CG': 8, 'GG': 7, 'GA': 2, 'TG': 3, '<pad>': 17, 'CT': 1, 'AT': 16, 'GT': 10, 'CA': 12, 'AC': 5, '<unk>': 0}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  250


In [9]:
tokenizer(train_dset[1][0])

['CC',
 'AT',
 'CT',
 'AT',
 'TT',
 'TT',
 'GA',
 'AT',
 'CC',
 'TT',
 'TT',
 'AA',
 'CA',
 'CT',
 'TT',
 'AT',
 'GA',
 'GA',
 'GA',
 'GA',
 'AA',
 'GA',
 'AC',
 'AG',
 'AA',
 'CA',
 'GG',
 'CA',
 'GG',
 'CA',
 'GG',
 'CT',
 'GT',
 'GT',
 'TT',
 'AA',
 'CC',
 'CC',
 'AC',
 'AA',
 'CA',
 'CC',
 'AC',
 'CA',
 'TA',
 'GA',
 'GC',
 'CT',
 'AA',
 'AA',
 'GC',
 'TT',
 'GG',
 'GA',
 'GT',
 'AA',
 'AT',
 'AC',
 'AA',
 'TT',
 'TC',
 'TT',
 'TG',
 'GC',
 'AC',
 'CT',
 'TG',
 'TA',
 'AT',
 'AA',
 'AA',
 'CA',
 'TG',
 'AA',
 'CA',
 'AT',
 'GA',
 'GT',
 'TT',
 'CT',
 'CA',
 'CC',
 'AT',
 'GT',
 'TA',
 'TT',
 'AG',
 'TC',
 'TC',
 'TG',
 'GA',
 'TT',
 'CC',
 'AC',
 'GC',
 'TC',
 'CG',
 'GC',
 'AT',
 'AC',
 'TT',
 'AC',
 'AC',
 'AT',
 'TT',
 'AC',
 'AG',
 'AA',
 'AA',
 'GA',
 'GA',
 'GA',
 'CA',
 'GA',
 'CA',
 'AA',
 'GA',
 'GA',
 'GT',
 'AC',
 'TG',
 'TG',
 'CT',
 'GA',
 'AA',
 'GG',
 'TT',
 'CA',
 'GA',
 'CA',
 'GA',
 'AG',
 'AA',
 'TT',
 'CG',
 'GG',
 'AC',
 'TC',
 'AA',
 'GC',
 'CA',
 'CA',
 'AC',

## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 62.9%, Avg loss: 0.653756 

Epoch 1


Train metrics: 
 Accuracy: 67.0%, Avg loss: 0.648485 

Epoch 2


Train metrics: 
 Accuracy: 67.3%, Avg loss: 0.640385 

Epoch 3


Train metrics: 
 Accuracy: 68.6%, Avg loss: 0.638475 

Epoch 4


Train metrics: 
 Accuracy: 70.8%, Avg loss: 0.632267 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  3474 ; tp  1866.2110809087753 ; fp  682.9774398999361
recall  0.5371937481026987 ; precision  0.7320804505728488
num_batches 218
correct 4701
size 6948
Test metrics: 
 Accuracy: 0.676598, F1 score: 0.619675, Avg loss: 0.647931 



(0.6765975820379966, 0.6196754673912169)